<h3>Загрузка датасету та бібліотек</h3>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv('Wine.csv')
df.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

In [12]:
df['Customer_Segment'] = df['Customer_Segment'] - 1
print(df['Customer_Segment'].unique())

[0 1 2]


<h3>Розбиття на тестову та тренувальну вибірки</h3>

In [13]:
# X - ознаки
X = df[['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium', 'Total_Phenols', 'Flavanoids',
        'Nonflavanoid_Phenols', 'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline']]

# y - цільова змінна
y = df['Customer_Segment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

<h3>Формування двох моделей(RandomForestClassifier, XGBClassifier)<br>Та зрівнення ефективності</h3>

In [14]:
# Створення двох одиночних моделей (в даному випадку, випадковий ліс та XGBoost)
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_xgb = XGBClassifier(n_estimators=100, random_state=42)

# Тренування одиночних моделей
model_rf.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)

# Оцінка точності одиночних моделей на тестовому наборі
y_pred_rf = model_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Точність випадкового лісу: {accuracy_rf}")

y_pred_xgb = model_xgb.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"Точність XGBoost: {accuracy_xgb}")

Точність випадкового лісу: 0.9722222222222222
Точність XGBoost: 0.9444444444444444


<h3>Створення паралелльного гомогенного ансамблю та послідовного, після чого зрівнення результатів ефективності</h3>

In [15]:
# Створення паралельного гомогенного ансамблю
ensemble_parallel = VotingClassifier(estimators=[('rf', model_rf), ('xgb', model_xgb)], voting='hard')
ensemble_parallel.fit(X_train, y_train)

# Оцінка точності паралельного гомогенного ансамблю на тестовому наборі
y_pred_ensemble_parallel = ensemble_parallel.predict(X_test)
accuracy_ensemble_parallel = accuracy_score(y_test, y_pred_ensemble_parallel)
print(f"Точність паралельного гомогенного ансамблю: {accuracy_ensemble_parallel}")

# Створення послідовного гомогенного ансамблю
ensemble_sequential = VotingClassifier(estimators=[('rf', model_rf), ('xgb', model_xgb)], voting='soft')
ensemble_sequential.fit(X_train, y_train)

# Оцінка точності послідовного гомогенного ансамблю на тестовому наборі
y_pred_ensemble_sequential = ensemble_sequential.predict(X_test)
accuracy_ensemble_sequential = accuracy_score(y_test, y_pred_ensemble_sequential)
print(f"Точність послідовного гомогенного ансамблю: {accuracy_ensemble_sequential}")

Точність паралельного гомогенного ансамблю: 0.9444444444444444
Точність послідовного гомогенного ансамблю: 0.9722222222222222


<h3>Створення паралельного простого ансамблю MaxVoting та складного Stacking (за допомогою LogisticRegression)</h3>

In [16]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [17]:
model_rf1 = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf2 = RandomForestClassifier(n_estimators=100, random_state=42)

# Простий метод: Max Voting (Паралельний)
ensemble_max_voting = VotingClassifier(estimators=[('rf1', model_rf1), ('rf2', model_rf2)], voting='hard')

# Тренування ансамблю Max Voting
ensemble_max_voting.fit(X_train, y_train)

# Прогноз на тестовому наборі
y_pred_max_voting = ensemble_max_voting.predict(X_test)

# Оцінка точності Max Voting
accuracy_max_voting = accuracy_score(y_test, y_pred_max_voting)
print(f"Точність Max Voting: {accuracy_max_voting}")

# Складний метод: Stacking (Паралельний)
# Створення базових моделей
base_models = [('rf1', model_rf1), ('rf2', model_rf2)]

# Створення моделі Stacking з використанням Logistic Regression в якості метамоделі
meta_model = LogisticRegression()
ensemble_stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Тренування ансамблю Stacking
ensemble_stacking.fit(X_train, y_train)

# Прогноз на тестовому наборі
y_pred_stacking = ensemble_stacking.predict(X_test)

# Оцінка точності Stacking
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
print(f"Точність Stacking: {accuracy_stacking}")

Точність Max Voting: 0.9722222222222222
Точність Stacking: 1.0


<h3>Підбір оптимальних параметрів для створення ансамблевих моделей</h3>

In [19]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV

# Створення двох одиночних моделей
model_rf1 = RandomForestClassifier(random_state=42)
model_rf2 = RandomForestClassifier(random_state=42)

# Створення гомогенного ансамблю
ensemble_max_voting = VotingClassifier(estimators=[('rf1', model_rf1), ('rf2', model_rf2)], voting='hard')

# Параметри для налаштування VotingClassifier
param_grid_voting = {
    'voting': ['hard', 'soft'],
    'weights': [[1, 1], [2, 1], [1, 2]]
}

# Перехресна перевірка для Max Voting (Паралельний)
grid_search_max_voting = GridSearchCV(ensemble_max_voting, param_grid_voting, cv=5)
grid_search_max_voting.fit(X_train, y_train)

# Оптимальні параметри для Max Voting
best_params_max_voting = grid_search_max_voting.best_params_
print(f"Оптимальні параметри для Max Voting: {best_params_max_voting}")


Оптимальні параметри для Max Voting: {'voting': 'hard', 'weights': [1, 1]}


<h3>Розробка та реалізуція n-рівневого гомогенного ансамблю (де n>2) з застосуванням VotingClassifier</h3>

In [22]:
# 1-й рівень: Багатокласовий класифікатор (Random Forest)
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Прогноз на тестовому наборі для 1-го рівня
y_pred_rf = model_rf.predict(X_test)

# Оцінка точності для 1-го рівня
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Точність 1-го рівня (Random Forest): {accuracy_rf}")

# 2-й рівень: Голосування (паралельний)
ensemble_voting = VotingClassifier(estimators=[('rf', model_rf)], voting='hard')
ensemble_voting.fit(X_train, y_train)

# Прогноз на тестовому наборі для 2-го рівня
y_pred_voting = ensemble_voting.predict(X_test)

# Оцінка точності для 2-го рівня
accuracy_voting = accuracy_score(y_test, y_pred_voting)
print(f"Точність 2-го рівня (Voting): {accuracy_voting}")

# 3-й рівень: Зважування (паралельний)
# За припущенням, що model_rf - це ваш Random Forest
ensemble_weighted = VotingClassifier(estimators=[('rf', model_rf)], voting='soft', weights=[2])
ensemble_weighted.fit(X_train, y_train)

# Прогноз на тестовому наборі для 3-го рівня
y_pred_weighted = ensemble_weighted.predict(X_test)

# Оцінка точності для 3-го рівня
accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
print(f"Точність 3-го рівня (Weighted): {accuracy_weighted}")

Точність 1-го рівня (Random Forest): 0.9722222222222222
Точність 2-го рівня (Voting): 0.9722222222222222
Точність 3-го рівня (Weighted): 0.9722222222222222
